In [3]:
import os
import dotenv

dotenv.load_dotenv()

CONNECTION = os.getenv("SQLALCHEMY_STOCKS1DAY_URI")
TWELVEDATA_KEY = os.getenv("TWELVEDATA_API_KEY")


from sqlalchemy import create_engine, text

engine = create_engine(CONNECTION)
with engine.connect() as conn:
    cursor = conn.execute(text("SELECT name FROM sqlite_master WHERE type='table';"))
    print([name[0] for name in cursor].__len__())

10219


In [2]:
import requests
# stocks list
STOCK_LIST_URL = "https://api.twelvedata.com/stocks?exchange={}&apikey={}"
NYSE_STOCK_LIST_RESPONSE: dict = requests.get(STOCK_LIST_URL.format("NYSE", TWELVEDATA_KEY)).json()
NASDAQ_STOCK_LIST_RESPONSE: dict = requests.get(STOCK_LIST_URL.format("NASDAQ", TWELVEDATA_KEY)).json()
stock_symbols = set(stock["symbol"] for stock in NYSE_STOCK_LIST_RESPONSE["data"] if stock["symbol"].isalnum()) | set(stock["symbol"] for stock in NASDAQ_STOCK_LIST_RESPONSE["data"] if stock["symbol"].isalnum())
print("Num stocks: ", stock_symbols.__len__())
# etf list
ETF_LIST_URL = "https://api.twelvedata.com/etfs?exchange={}&apikey={}"
NYSE_ETF_LIST_RESPONSE: dict = requests.get(ETF_LIST_URL.format("NYSE", TWELVEDATA_KEY)).json()
NASDAQ_ETF_LIST_RESPONSE: dict = requests.get(ETF_LIST_URL.format("NASDAQ", TWELVEDATA_KEY)).json()
etf_symbols = set(stock["symbol"] for stock in NYSE_ETF_LIST_RESPONSE["data"] if stock["symbol"].isalnum()) | set(stock["symbol"] for stock in NASDAQ_ETF_LIST_RESPONSE["data"] if stock["symbol"].isalnum())
print("Num etfs: ", etf_symbols.__len__())

engine = create_engine(CONNECTION)
existing_tables = ()
with engine.connect() as conn:
    cursor = conn.execute(text("SELECT name FROM sqlite_master WHERE type='table';"))
    existing_tables = set([name[0] for name in cursor])

symbols = list((stock_symbols | etf_symbols) - existing_tables)
print("Remaining symbols: ", symbols.__len__())

Num stocks:  7252
Num etfs:  3065
Remaining symbols:  99


In [4]:
import pandas as pd
from sqlalchemy import create_engine
from scipy.stats import entropy, gaussian_kde, norm
import numpy as np

df = pd.read_sql_table("SPY", engine, index_col="timestamp")

print("Log price subtraction:", np.log(df["close"][0] / df["close"][251]))
print("Regular Return:", (df["close"][0] - df["close"][251]) / df["close"][251])
print("Log sum:", np.sum(df["log_return"][:251]))
print("Reg sum:", np.sum(df["return"][:251]))
print("Annualized return:", (df["close"][0] / df["close"][251])**(1/251-1))




Log price subtraction: 0.19434698958331015
Regular Return: 0.21451763481768021
Log sum: 0.19434698958331026
Reg sum: 0.20239356110270335
Annualized return: 0.824009929494261


/tmp/ipykernel_58759/1465022030.py:8: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("Log price subtraction:", np.log(df["close"][0] / df["close"][251]))
/tmp/ipykernel_58759/1465022030.py:9: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("Regular Return:", (df["close"][0] - df["close"][251]) / df["close"][251])
/tmp/ipykernel_58759/1465022030.py:12: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print("Annualized return:",

In [5]:
from scipy.stats import gaussian_kde
from scipy.integrate import quad
annual_returns = df["log_return"][::-1].rolling(window=251).sum().dropna()
print(annual_returns)
kde = gaussian_kde(annual_returns)
num_points = 1000
possible_returns = np.linspace(-1, 1, num_points)
densities = kde.pdf(possible_returns)
total_density = np.sum(densities)
probabilities = densities/total_density
np.sum(possible_returns * probabilities)

timestamp
2006-04-03    0.089699
2006-04-04    0.096076
2006-04-05    0.094135
2006-04-06    0.103520
2006-04-07    0.092543
                ...   
2025-02-10    0.187975
2025-02-11    0.189175
2025-02-12    0.199817
2025-02-13    0.201272
2025-02-14    0.194347
Name: log_return, Length: 4750, dtype: float64


0.07955745250589819

In [11]:
def calculate_expected_return(df: pd.DataFrame) -> float:
    # get rolling annual returns at every day
    annual_returns = df["return"][::-1].rolling(window=251).apply(lambda x: np.prod(x+1)-1).dropna()
    # estimate the PDF using gaussian kernel density estimation
    try:
        kde = gaussian_kde(annual_returns)
    except:
        return -1
    # get a range of 1000 discrete possible returns,
    # bounding daily returns between 100% and -100%
    possible_returns = np.linspace(-1, 1, 1000)
    # get density estimates for each possible return
    densities = kde.pdf(possible_returns)
    # get the expected return by multiplying each possible return by its probability
    # then summing over all possible returns
    return np.sum(possible_returns * (densities/np.sum(densities)))

0.09656232560810077

In [ ]:
from pypfopt import black_litterman
from pypfopt.black_litterman import BlackLittermanModel
from pypfopt.efficient_frontier import EfficientFrontier